In [2]:
# STATIC_LOADER
import pandas as pd


class STATUS_LOADER:
    """
    STATUS_LOADER : 상태 정보 추출 클래스
    """

    def __init__(self, dict_df_result: dict, dict_df_position: dict) -> None:
        """
        STATUS_LOADER의 생성자

        :param dict dict_df_result: dict_df_result 입니다.
        :param dict dict_df_position: dict_df_position 입니다.
        :return: None
        :rtype: None
        """
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self) -> float:
        """
        현재 보유 cash를 반환하는 메서드

        :return : 현재 보유 cash
        :rtype: float
        """
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000.0

    def get_status_df(self) -> pd.DataFrame:
        """
        현재 보유 position 관련 정보를 반환하는 메서드

        :return: 보유한 symbol, 갯수, 구매가격, 현재 가격을 데이터프레임 형태로 가져옵니다.
        :rtype: pd.DataFrame
        """
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "SYMBOL": _symbol,
                        "CURRENT_QTY": _current_qty,
                        "CURRENT_PRICE": _current_price,
                        "TRADE_PRICE": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(
            current_symbol_list,
            columns=["SYMBOL", "CURRENT_QTY", "CURRENT_PRICE", "TRADE_PRICE"],
        )

In [3]:
# API_LOADER
import datetime as dt

import pandas as pd
import kquant as kq


class SYMBOL_LOADER:
    """
    SYMBOL_LOADER : 거래가능한 주식 symbol을 필터-추출하는 클래스
    """

    @staticmethod
    def load_symbols_df() -> pd.DataFrame:
        """
        한국거래소 종목 목록 데이터프레임을 호출하는 메서드

        :return : 한국거래소 종목 목록 데이터프레임
        :rtype : pd.DataFrame
        """
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        """
        SYMBOl_FILTER : symbols를 filtering 하는 클래스
        """

        @staticmethod
        def filter__market(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            market에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: MARKET이 [코스닥, 유가증권]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            ADMIN_ISSUE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: ADMIN_ISSUE가 0인 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            SEC_TYPE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: SEC_TYPE이 [ST, EF, EN]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df: pd.DataFrame) -> pd.DataFrame:
        """
        symbol_df 에 대한 필터링을 진행하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: SYMBOL_FILTER의 메서드를 거친 데이터프레임
        :rtype: pd.DataFrame
        """
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df: pd.DataFrame) -> list:
        """
        symbols_df의 symbol을 중복을 제거하여 추출하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: symbols
        :rtype: list
        """
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self) -> list:
        """
        SYMBOL_LOADER의 파이프라인을 제공하는 메서드

        :return: 필터를 거친 symbols
        :rtype: list
        """
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols


class FUNDAMENTAL_LOADER:
    """
    FUNDAMENTAL_LOADER : fundamental_analysis를 위한 정보를 추출하는 클래스
    """

    def __init__(self, symbol: str, date: dt.date) -> None:
        """
        FUNDAMENTAL_LOADER의 생성자

        :param str symbol: stock의 symbol 입니다.
        :param datetime.date date: 현재 날짜 입니다.

        :attr : pd.DataFrmae daily_stock_df : 현재 날짜 기준 가장 최근 stock데이터 입니다.
        """
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self) -> float:
        """
        가장 최근 종가를 추출합니다.
        :return: 종가
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self) -> float:
        """
        가장 최근 시가총액을 추출합니다.

        :return: 시가총액
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return float(_marketcap)

    def load_recent_netprofit(self) -> float:
        """
        공시자료 중 가장 최근 당기순이익을 추출합니다.

        :return: 당기순이익
        :rtype: float
        """
        netprofit_df = kq.account_history(
            symbol=self.symbol, account_code="122700", period="q"
        )
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0] * 1000
        return float(_netprofit)

    def load_recent_assets(self) -> float:
        """
        공시자료 중 가장 최근 총 자산를 추출합니다.

        :return: 총 자산
        :rtype: float
        """
        assets_df = kq.account_history(
            symbol=self.symbol, account_code="111000", period="q"
        )
        assets_df.sort_values("YEARMONTH", inplace=True)
        _assets = assets_df.tail(1)["VALUE"].values[0] * 1000
        return float(_assets)

    def load_recent_current_assets(self) -> float:
        """
        공시자료 중 가장 최근 유동 자산를 추출합니다.

        :return: 유동 자산
        :rtype: float
        """
        current_assets_df = kq.account_history(
            symbol=self.symbol, account_code="111100", period="q"
        )
        current_assets_df.sort_values("YEARMONTH", inplace=True)
        _current_assets = current_assets_df.tail(1)["VALUE"].values[0] * 1000
        return float(_current_assets)

    def load_recent_liabilities(self) -> float:
        """
        공시자료 중 가장 최근 총 부채를 추출합니다.

        :return: 총 부채
        :rtype: float
        """
        liabilities_df = kq.account_history(
            symbol=self.symbol, account_code="113000", period="q"
        )
        liabilities_df.sort_values("YEARMONTH", inplace=True)
        _liabilities = liabilities_df.tail(1)["VALUE"].values[0] * 1000
        return float(_liabilities)

    def load_recent_equity(self) -> float:
        """
        공시자료 중 가장 최근 총 자본(총 자산 - 총 부채)를 추출합니다.

        :return: 총 자본(총 자산 - 총 부채)
        :rtype: float
        """
        equity_df = kq.account_history(
            symbol=self.symbol, account_code="115000", period="q"
        )
        equity_df.sort_values("YEARMONTH", inplace=True)
        _equity = equity_df.tail(1)["VALUE"].values[0] * 1000
        return float(_equity)

    def load_recent_EBITDA(self) -> float:
        """
        공시자료 중 가장 최근 EBITDA를 추출합니다.

        :return: EBITDA
        :rtype: float
        """
        ebitda_df = kq.account_history(
            symbol=self.symbol, account_code="123000", period="q"
        )
        ebitda_df.sort_values("YEARMONTH", inplace=True)
        _ebitda = ebitda_df.tail(1)["VALUE"].values[0] * 1000
        return float(_ebitda)

    def __call__(self) -> dict:
        """
        fundmanetal analysis를 위해 필요한 데이터를 가져와서 dictionary를 반환한다.

        :return: fundamental analysis를 위한 데이터 dictionary
        :rtype: dict
        """
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _assets = self.load_recent_assets()
        _equity = self.load_recent_equity()
        return {
            "SYMBOL": self.symbol,
            "CLOSE": _close,
            "MARKETCAP": _marketcap,
            "NETPROFIT": _netprofit,
            "ASSETS": _assets,
            "EQUITY": _equity,
        }


In [4]:
# MODEL_PROCESSOR
import pandas as pd


class PBR_PROCESSOR:
    """
    PBR_PROCESSOR : PBR에 대한 정보를 SCORE로 정제하여 반환하는 클래스
    """

    def __init__(self, fundamental_df) -> None:
        self.fundamental_df = fundamental_df

    @staticmethod
    def append_pbr(fundamental_df):
        """
        PBR을 추가하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PBR가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PBR"] = fundamental_df["MARKETCAP"] / (fundamental_df["EQUITY"])
        return fundamental_df

    @staticmethod
    def filter_negative_pbr(fundamental_df):
        """
        PBR이 음수인 데이터를 제거하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PBR이 음수가 아닌 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = fundamental_df[fundamental_df["PBR"] > 0]
        return fundamental_df

    @staticmethod
    def append_score(fundamental_df: pd.DataFrame) -> pd.DataFrame:
        """
        PBR을 기준으로, 낮은 PER일 수록 큰 SCORE를 주는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PBR_SCORE"] = (
            fundamental_df["PBR"].sum() / fundamental_df["PBR"]
        )
        return fundamental_df

    def __call__(self):
        """
        PBR기반의 score 추가한 데이터프레임을 반환하는 메서드

        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = self.fundamental_df

        fundamental_df = self.append_pbr(fundamental_df)
        fundamental_df = self.filter_negative_pbr(fundamental_df)
        fundamental_df = self.append_score(fundamental_df)

        score_df = fundamental_df.loc[:, ["SYMBOL", "PBR_SCORE"]]
        return score_df


class PER_PROCESSOR:
    """
    PER_PROCESSOR : PER에 대한 정보를 SCORE로 정제하여 반환하는 클래스
    """

    def __init__(self, fundamental_df) -> None:
        """
        PER_PROCESSOR의 생성자

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        """
        self.fundamental_df = fundamental_df

    @staticmethod
    def append_per(fundamental_df):
        """
        PER을 추가하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PER가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PER"] = fundamental_df["MARKETCAP"] / (
            fundamental_df["NETPROFIT"]
        )
        return fundamental_df

    @staticmethod
    def filter_negative_per(fundamental_df):
        """
        PER이 음수인 데이터를 제거하는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: PER이 음수가 아닌 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = fundamental_df[fundamental_df["PER"] > 0]
        return fundamental_df

    @staticmethod
    def append_score(fundamental_df: pd.DataFrame) -> pd.DataFrame:
        """
        PER을 기준으로, 낮은 PER일 수록 큰 SCORE를 주는 메서드

        :param pd.DataFrame fundamental_df: fundamental 데이터를 가지고 있는 데이터프레임
        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df["PER_SCORE"] = (
            fundamental_df["PER"].sum() / fundamental_df["PER"]
        )
        return fundamental_df

    def __call__(self):
        """
        PER기반의 score 추가한 데이터프레임을 반환하는 메서드

        :return: SCORE가 추가된 fundamental_df
        :rtype: pd.DataFrame
        """
        fundamental_df = self.fundamental_df

        fundamental_df = self.append_per(fundamental_df)
        fundamental_df = self.filter_negative_per(fundamental_df)
        fundamental_df = self.append_score(fundamental_df)

        score_df = fundamental_df.loc[:, ["SYMBOL", "PER_SCORE"]]
        return score_df


In [5]:
# ORDER_PROCESSOR

import numpy as np
import pandas as pd


class BUYING_ORDER_PROCESSOR:
    """
    BUYING_ORDER_PROCESSOR : 매수주문을 생성하는 클래스
    """

    def __init__(self, score_df, invest_money, status_df, n) -> None:
        """
        BUYING_ORDER_PROCESSOR의 생성자

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        self.score_df = score_df
        self.invest_money = invest_money
        self.status_df = status_df
        self.n = n

    @staticmethod
    def filter_positioned_symbol(score_df, positioned_symbol):
        """
        이미 position이 있는 종목을 필터링 하는 메소드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param list positioned_symbol: 현재 이미 position이 있는 symbol의 리스트
        """
        filtered_score_df = score_df[~(score_df["SYMBOL"].isin(positioned_symbol))]
        return filtered_score_df

    @staticmethod
    def get_high_score_df(score_df, n):
        """
        score_df의 상위 n개의 row를 추출하는 메서드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        high_score_df = score_df.nlargest(n, "SCORE")
        return high_score_df

    @staticmethod
    def append_price_invest(high_score_df, invest_money):
        """
        당일 활용 투자 금액을 score 기준으로 분배한 column을 생성하는 메서드

        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        """
        high_score_df["PRICE_INVEST"] = (
            high_score_df["SCORE"] / high_score_df["SCORE"].sum()
        ) * invest_money
        return high_score_df

    @staticmethod
    def append_cnt_invest(high_score_df: pd.DataFrame) -> pd.DataFrame:
        """
        당일 활용 투자 금액 최근 종가로 나누어 주문 갯수를 column으로 생성하는 메서드

        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        """
        high_score_df["CNT_INVEST"] = (
            high_score_df["PRICE_INVEST"] // high_score_df["CLOSE"]
        )
        return high_score_df

    @staticmethod
    def get_order_from_df(df):
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드

        :param pd.DataFrame df: [SYMBOL,CNT_INVEST]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CNT_INVEST"].astype(int).to_dict().items()
        )
        return orders

    def __call__(self) -> list:
        """
        BUYING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드

        """
        score_df = self.score_df
        invest_money = self.invest_money
        status_df = self.status_df
        n = self.n

        positioned_symbol = sorted(set(status_df["SYMBOL"]))
        filtered_positioned_df = self.filter_positioned_symbol(
            score_df, positioned_symbol
        )

        high_score_df = self.get_high_score_df(filtered_positioned_df, n)
        high_score_df = self.append_price_invest(high_score_df, invest_money)
        high_score_df = self.append_cnt_invest(high_score_df)
        buying_order = self.get_order_from_df(high_score_df)
        return buying_order


class SELLING_ORDER_PROCESSOR:
    """
    SELLING_ORDER_PROCESSOR : 매도주문을 추출하는 클래스
    """

    def __init__(self, score_df, status_df, percentage) -> None:
        """
        SELLING_ORDER_PROCESSOR의 생성자

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param int percentage: low_score를 선정하기 위한 하위 (n-percent)의 기준
        """
        self.score_df = score_df
        self.status_df = status_df
        self.percentage = percentage

    @staticmethod
    def get_limit_line(score_df, percentage):
        """
        score 하위 [percentage]% 를 한계선으로 설정하고 이를 기준값을 추출

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param int percentage: low_score를 선정하기 위한 하위 (n-percent)의 기준
        """
        limit_line = np.percentile(score_df["SCORE"], percentage)
        return limit_line

    @staticmethod
    def get_low_score_df(score_df: pd.DataFrame, limit_line: float) -> pd.DataFrame:
        """
        기준값을 바탕으로 low_score_df를 추출하는 메서드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param float limit_line: get_limit_line에서 추출한 하위 한계선
        """
        low_score_df = score_df[score_df["SCORE"] < limit_line]
        return low_score_df

    @staticmethod
    def get_filtered_position_symbols(
        low_score_df: pd.DataFrame, position_symbols: list
    ) -> pd.DataFrame:
        """
        low_score_df 중 position_symbol에 속하는 symbol만 추출하는 메서드

        :param pd.DataFrame low_score_df: symbol,score,close를 가진 데이터프레임
        :param list position_symbols: 현재 이미 position이 있는 symbol의 리스트
        """
        filtered_position_symbols = sorted(
            set(low_score_df["SYMBOL"]) & set(position_symbols)
        )
        return filtered_position_symbols

    @staticmethod
    def get_order_from_df(df):
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드

        :param pd.DataFrame df: [SYMBOL,CURRENT_QTY]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CURRENT_QTY"]
            .apply(lambda x: x * -1)
            .astype(int)
            .to_dict()
            .items()
        )
        return orders

    def __call__(self) -> list:
        """
        SELLING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드
        """
        score_df = self.score_df
        status_df = self.status_df
        percentage = self.percentage
        position_symbols = sorted(set(status_df["SYMBOL"]))

        limit_line = self.get_limit_line(score_df, percentage)
        low_score_df = self.get_low_score_df(score_df, limit_line)
        filtered_position_symbols = self.get_filtered_position_symbols(
            low_score_df, position_symbols
        )

        selling_df = status_df[status_df["SYMBOL"].isin(filtered_position_symbols)]
        selling_orders = self.get_order_from_df(selling_df)

        return selling_orders


In [6]:
# TRADE_FUNC
import random
import logging
import datetime as dt

import pandas as pd
import kquant as kq
from sklearn.preprocessing import MinMaxScaler


# from .loader.static_loader import STATUS_LOADER
# from .loader.api_loader import SYMBOL_LOADER, FUNDAMENTAL_LOADER
# from .processor.model_processor import PBR_PROCESSOR, PER_PROCESSOR
# from .processor.order_processor import BUYING_ORDER_PROCESSOR, SELLING_ORDER_PROCESSOR


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    """
    CFG : trade_fun의 파라미터 조정

    """
    CFG = {
        "cash_percentage": 0.75,  # 1일 투자 금액 (보유 현금 * 0.75)
        "pbr_ratio": 1,  # pbr_score 가중치
        "per_ratio": 0.3,  # per_score 가중치
        "sampling_size": 230,  # 전체 주식중 sampling 사이즈
        "buying_order_n": 10,  # 1일 구매 stock 종류수
        "selling_order_percentage": 30,  # score 하위 40프로 이하면 매도
    }

    """
    STATUS_LOADER
    """
    status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

    current_cash = status_loader.get_current_cash()
    invest_money = current_cash * CFG["cash_percentage"]

    status_df = status_loader.get_status_df()
    position_symbols = sorted(set(status_df["SYMBOL"]))

    """
    SYMBOL_LOADER
    """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    sampled_symbols = random.sample(total_symbols, CFG["sampling_size"])
    using_symbols = sorted(set(sampled_symbols + position_symbols))

    """
    FUNDAMENTAL_LOADER
    """
    fundamental_data_list = list()
    for symbol in using_symbols:
        try:
            _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
            _fundamental_data = _fundamental_loader()
            fundamental_data_list.append(_fundamental_data)
        except:
            pass
    fundamental_df = pd.DataFrame(fundamental_data_list)
    symbol_close_dict = fundamental_df.set_index("SYMBOL")["CLOSE"].to_dict()

    """
    PBR_PROCESSOR
    """
    pbr_processor = PBR_PROCESSOR(fundamental_df)
    pbr_score_df = pbr_processor()

    """
    PER_PROCESSOR
    """
    per_processor = PER_PROCESSOR(fundamental_df)
    per_score_df = per_processor()

    """
    CONCAT SCORES
    """
    score_df = pbr_score_df.merge(
        per_score_df.loc[:, ["SYMBOL", "PER_SCORE"]], on="SYMBOL"
    )

    mms = MinMaxScaler()
    score_df.iloc[:, 1:] = mms.fit_transform(score_df.iloc[:, 1:])

    score_df["SCORE"] = (
        score_df["PBR_SCORE"] * CFG["pbr_ratio"]
        + score_df["PER_SCORE"] * CFG["per_ratio"]
    )
    score_df["CLOSE"] = score_df["SYMBOL"].map(symbol_close_dict)

    """
    BUYING_ORDER_PROCESSOR
    """
    buying_order_processor = BUYING_ORDER_PROCESSOR(
        score_df, invest_money, status_df, CFG["buying_order_n"]
    )
    buying_orders = buying_order_processor()

    """
    SELLING_ORDER_PROCESSOR
    """
    selling_order_processor = SELLING_ORDER_PROCESSOR(
        score_df, status_df, CFG["selling_order_percentage"]
    )
    selling_orders = selling_order_processor()

    symbols_and_orders = buying_orders + selling_orders
    return symbols_and_orders

In [ ]:
import datetime as dt
dict_df_result_1, dict_df_position_1, logger = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-22",
    "2023-08-22",
    init_cash=1_000_000_000,
    return_position=True,
    return_logger=True,
)

In [ ]:
date = dt.date(2023, 8, 23)
symbols_and_orders = trade_func(
    date,
    dict_df_result_1,
    dict_df_position_1,
    logger,
)
dict_df_result_2, dict_df_position_2 = kq.backtest_update_stock_port_daily(
    symbols_and_orders,
    date,
    dict_df_result_1,
    dict_df_position_1,
)

In [ ]:
date = dt.date(2023, 8, 24)

symbols_and_orders = trade_func(
    date,
    dict_df_result_2,
    dict_df_position_2,
    logger,
)

dict_df_result_3, dict_df_position_3 = kq.backtest_update_stock_port_daily(
    symbols_and_orders,
    date,
    dict_df_result_2,
    dict_df_position_2,
)

In [ ]:
date = dt.date(2023, 9, 22)

dict_df_result_4, dict_df_position_4 = kq.backtest_update_stock_port_daily(
    [],
    date,
    dict_df_result_3,
    dict_df_position_3,
)
result = dict_df_result_4["TOTAL"]["TOTAL_VALUE"].tail(1).values[0]
result

In [ ]:
result

In [ ]:
import kquant as kq

# loop
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-22",  # 실제 심사에서는 투자기간 시작일
    "2023-09-10",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)